# 01 - LETTURA CARTELLA IMMAGINI PER POPOLARE IL DATABASE

#### 0) Lettura file con immagini

In [30]:
from pathlib import Path
import pandas as pd
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent   # notebooks → root progetto
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("✅ src aggiunto al PYTHONPATH:", SRC_PATH)

from garbagecls.paths import abs_path
from garbagecls.db import read_images_df, split_dfs
# -------------------------
# Paths (repo root robusto)
# -------------------------
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name.lower() == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

RAW_FLAT   = PROJECT_ROOT / "data" / "raw_flat"
EXTERNAL   = PROJECT_ROOT / "data" / "external"

ALLOWED_EXT = {".jpg", ".jpeg", ".png", ".webp"}

def scan_image_folder(root_dir: Path, source_name: str) -> pd.DataFrame:
    """Scansiona root_dir strutturata come root_dir/<label>/* e produce dataframe con filepath,label,source"""
    records = []
    if not root_dir.exists():
        print(f"⚠️ Cartella non trovata: {root_dir}")
        return pd.DataFrame(columns=["filepath","label","source"])

    for class_dir in root_dir.iterdir():
        if not class_dir.is_dir():
            continue
        label = class_dir.name

        for img_path in class_dir.rglob("*"):
            if not img_path.is_file():
                continue
            if img_path.suffix.lower() not in ALLOWED_EXT:
                continue

            records.append({
                "filepath": img_path.relative_to(PROJECT_ROOT).as_posix(),
                "label": label,
                "source": source_name
            })
    return pd.DataFrame(records)

# --- Scan main dataset ---
df_raw = scan_image_folder(RAW_FLAT, source_name="raw_flat")
print("RAW_FLAT images:", len(df_raw))
display(df_raw.head())
print(df_raw["label"].value_counts())

# --- Scan external (our photos) ---
df_external = scan_image_folder(EXTERNAL, source_name="external")
print("\nEXTERNAL images:", len(df_external))
if len(df_external) > 0:
    display(df_external.head())
    print(df_external["label"].value_counts())


✅ src aggiunto al PYTHONPATH: c:\Users\simon\Documents\VSC\Progetto-Data-Science\src
RAW_FLAT images: 2527


,filepath,label,source
0,data/raw_flat/cardboard/cardboard1.jpg,cardboard,raw_flat
1,data/raw_flat/cardboard/cardboard10.jpg,cardboard,raw_flat
2,data/raw_flat/cardboard/cardboard100.jpg,cardboard,raw_flat
3,data/raw_flat/cardboard/cardboard101.jpg,cardboard,raw_flat
4,data/raw_flat/cardboard/cardboard102.jpg,cardboard,raw_flat


label
paper        594
glass        501
plastic      482
metal        410
cardboard    403
trash        137
Name: count, dtype: int64

EXTERNAL images: 28


,filepath,label,source
0,data/external/cardboard/cardboard1.jpeg,cardboard,external
1,data/external/cardboard/cardboard2.jpeg,cardboard,external
2,data/external/cardboard/cardboard3.jpeg,cardboard,external
3,data/external/cardboard/cardboard4.jpeg,cardboard,external
4,data/external/cardboard/cardboard5.jpeg,cardboard,external


label
plastic      11
cardboard     7
glass         5
metal         2
paper         2
trash         1
Name: count, dtype: int64


### 1) Split stratificato da caricare poi su DB

In [31]:
from sklearn.model_selection import train_test_split

SEED = 42
TRAIN_SIZE = 0.70
VAL_SIZE = 0.15
TEST_SIZE = 0.15

# 1) train vs temp
df_train, df_temp = train_test_split(
    df_raw,
    test_size=(1 - TRAIN_SIZE),
    random_state=SEED,
    stratify=df_raw["label"]
)

# 2) val vs test (split del temp)
# nel "temp" abbiamo 30%: lo separiamo in metà e metà -> 15% e 15%
df_val, df_test = train_test_split(
    df_temp,
    test_size=TEST_SIZE / (VAL_SIZE + TEST_SIZE),
    random_state=SEED,
    stratify=df_temp["label"]
)

df_train = df_train.assign(split="train")
df_val   = df_val.assign(split="val")
df_test  = df_test.assign(split="test")

df_split = pd.concat([df_train, df_val, df_test], ignore_index=True)

print(df_split["split"].value_counts())
display(pd.crosstab(df_split["split"], df_split["label"]))


split
train    1768
test      380
val       379
Name: count, dtype: int64


label,cardboard,glass,metal,paper,plastic,trash
split,,,,,,
test,60,76,62,89,73,20
train,282,350,287,416,337,96
val,61,75,61,89,72,21


In [32]:
# =========================
# Add EXTERNAL to df_split
# =========================
# External non viene splittato: lo usiamo solo come test esterno (notebook 07).
# Mettiamo split='external' solo come etichetta informativa.
if df_external is not None and len(df_external) > 0:
    df_external2 = df_external.copy()
    df_external2["split"] = "test" #solo per coerenza con il codice esistente
    df_external2["source"] = "external"
    df_to_insert = pd.concat([df_split, df_external2], ignore_index=True)
else:
    df_to_insert = df_split

print("\nTotale righe da inserire:", len(df_to_insert))
display(pd.crosstab(df_to_insert["split"], df_to_insert["source"]))


Totale righe da inserire: 2555


source,external,raw_flat
split,,
test,28,380
train,0,1768
val,0,379


In [33]:
def filter_existing(df):
    df = df.copy()
    mask = df["filepath"].apply(lambda p: Path(str(abs_path(p))).exists())
    missing = df.loc[~mask, "filepath"].tolist()
    if missing:
        print(f"⚠️ Scartati {len(missing)} filepath non trovati su disco.")
        for p in missing[:10]:
            print(" -", p)
    return df.loc[mask].reset_index(drop=True)

df_to_insert = filter_existing(df_to_insert)
print("✅ Righe dopo filtro file esistenti:", len(df_to_insert))


✅ Righe dopo filtro file esistenti: 2555


In [34]:
df_to_insert["split"] = df_to_insert["split"].astype(str)
df_to_insert["source"] = df_to_insert["source"].astype(str)
df_to_insert["label"] = df_to_insert["label"].astype(str)
df_to_insert["filepath"] = df_to_insert["filepath"].astype(str)

In [35]:
print(df_train)

                                      filepath      label    source  split
2382       data/raw_flat/plastic/plastic92.jpg    plastic  raw_flat  train
229   data/raw_flat/cardboard/cardboard305.jpg  cardboard  raw_flat  train
352    data/raw_flat/cardboard/cardboard53.jpg  cardboard  raw_flat  train
340    data/raw_flat/cardboard/cardboard42.jpg  cardboard  raw_flat  train
951           data/raw_flat/metal/metal141.jpg      metal  raw_flat  train
...                                        ...        ...       ...    ...
2035      data/raw_flat/plastic/plastic213.jpg    plastic  raw_flat  train
709           data/raw_flat/glass/glass375.jpg      glass  raw_flat  train
2242       data/raw_flat/plastic/plastic40.jpg    plastic  raw_flat  train
610           data/raw_flat/glass/glass286.jpg      glass  raw_flat  train
974           data/raw_flat/metal/metal162.jpg      metal  raw_flat  train

[1768 rows x 4 columns]


#### 2) Connessione DB

In [36]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import execute_values

load_dotenv()

DB_CONFIG = dict(
    host=os.getenv("DB_HOST"),
    port=int(os.getenv("DB_PORT")),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
)

conn = psycopg2.connect(**DB_CONFIG)
conn.autocommit = True
cur = conn.cursor()
print("✅ Connected")


✅ Connected


In [37]:
from psycopg2.extras import execute_values

def cleanup_source(conn, source, valid_filepaths):
    """
    Rimuove dal DB le righe con source=... che non sono più presenti sul filesystem.
    valid_filepaths: lista/iterabile dei filepath "relativi" tipo 'data/external/...'
    """
    valid_filepaths = list(valid_filepaths)
    with conn.cursor() as cur:
        # Se non ci sono file, evita di cancellare tutto per errore
        if len(valid_filepaths) == 0:
            print(f"⚠️ cleanup_source: valid_filepaths vuoto per source='{source}'. Skip.")
            return

        cur.execute("""
            DELETE FROM images
            WHERE source = %s
              AND NOT (filepath = ANY(%s));
        """, (source, valid_filepaths))

        print(f"🧹 Cleanup DB: rimossi {cur.rowcount} record obsoleti per source='{source}'")
    conn.commit()

# costruisco le liste valide per ogni source
valid_raw = df_to_insert.loc[df_to_insert["source"]=="raw_flat", "filepath"]
valid_ext = df_to_insert.loc[df_to_insert["source"]=="external", "filepath"]

cleanup_source(conn, "raw_flat", valid_raw)
cleanup_source(conn, "external", valid_ext)


🧹 Cleanup DB: rimossi 0 record obsoleti per source='raw_flat'
🧹 Cleanup DB: rimossi 5 record obsoleti per source='external'


#### 3) Inserimento informazioni aggiuntive nel DB

In [38]:
rows = df_to_insert[["filepath", "label", "split", "source"]].to_records(index=False)

sql = """
INSERT INTO images (filepath, label, split, source)
VALUES %s
ON CONFLICT (filepath) DO UPDATE
SET label = EXCLUDED.label,
    split = EXCLUDED.split,
    source = EXCLUDED.source;
"""

execute_values(cur, sql, rows, page_size=1000)
print("✅ Insert/Upsert completato")


✅ Insert/Upsert completato


#### 4) Verifica

In [39]:
cur.execute("SELECT * FROM images")
cur.fetchall()[:10]


[(106,
  'data/raw_flat/trash/trash94.jpg',
  'trash',
  'train',
  'raw_flat',
  512,
  384,
  3,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (71,
  'data/raw_flat/paper/paper230.jpg',
  'paper',
  'train',
  'raw_flat',
  512,
  384,
  3,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 754164, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (1047,
  'data/raw_flat/cardboard/cardboard7.jpg',
  'cardboard',
  'train',
  'raw_flat',
  512,
  384,
  3,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 782033, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (1048,
  'data/raw_flat/glass/glass383.jpg',
  'glass',
  'train',
  'raw_flat',
  512,
  384,
  3,
  None,
  datetime.datetime(2026, 1, 30, 16, 42, 54, 782033, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))),
 (1049,
  'data/raw_flat/paper/paper593.jpg',
  'paper',
  'train',
  'raw_flat',
  512,
  384,
  

In [40]:
cur.close()
conn.close()


#### 5) Update DB con ultimi metadati: WIDTH / HEIGHT / CHANNELS

In [41]:
import os
from pathlib import Path
import psycopg2
from psycopg2.extras import RealDictCursor
from PIL import Image

# -------------------------
# 1) Project root (robusto)
# -------------------------
# Se il notebook sta in /notebooks, risali di 1 livello
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name.lower() == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

# -------------------------
# 2) Connessione DB (da .env)
# -------------------------
conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "127.0.0.1"),
    port=int(os.getenv("DB_PORT", "5432")),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
)

def get_whc(abs_path: Path):
    with Image.open(abs_path) as im:
        w, h = im.size
        # channels robusto
        c = len(im.getbands())
        return w, h, c

# -------------------------
# 3) Leggo solo righe da completare
# -------------------------
with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute("""
        SELECT image_id, filepath
        FROM images
        WHERE width IS NULL OR height IS NULL OR channels IS NULL
    """)
    rows = cur.fetchall()

print(f"Da completare: {len(rows)} immagini")

# -------------------------
# 4) Calcolo width/height/channels
# -------------------------
updates = []
errors = 0

for r in rows:
    image_id = r["image_id"]
    rel = r["filepath"]             # es: data/raw_flat/cardboard/cardboard2.jpg
    abs_path = (PROJECT_ROOT / Path(rel)).resolve()

    try:
        w, h, c = get_whc(abs_path)
        updates.append((w, h, c, image_id))
    except Exception as e:
        errors += 1
        # log minimale utile
        print(f"ERRORE image_id={image_id} path={rel} -> {e}")

print(f"Pronte {len(updates)} updates, errori {errors}")

# -------------------------
# 5) Update batch
# -------------------------
with conn:
    with conn.cursor() as cur:
        cur.executemany("""
            UPDATE images
            SET width = %s, height = %s, channels = %s
            WHERE image_id = %s;
        """, updates)

conn.close()
print("✅ Aggiornamento completato")


Da completare: 0 immagini
Pronte 0 updates, errori 0
✅ Aggiornamento completato
